# Estimate Binomial draw probabilility

In [1]:
using DynamicHMCModels

Define a structure to hold the data.

In [2]:
struct BernoulliProblem
    "Total number of draws in the data."
    n::Int
    "Number of draws `==1` in the data"
    s::Vector{Int}
end;

Make the type callable with the parameters *as a single argument*.

In [3]:
function (problem::BernoulliProblem)((α, )::NamedTuple{(:α, )})
    @unpack n, s = problem        # extract the data
    loglikelihood(Binomial(n, α), s)
end

Create the data and complete setting up the problem.

In [4]:
obs = rand(Binomial(9, 2/3), 1)
p = BernoulliProblem(9, obs)
p((α = 0.5, ))

-1.8075078261961934

Write a function to return properly dimensioned transformation.

In [5]:
problem_transformation(p::BernoulliProblem) =
    as((α = as𝕀, ),  )
# Use a flat priors (the default, omitted) for α
P = TransformedLogDensity(problem_transformation(p), p)
∇P = LogDensityRejectErrors(ADgradient(:ForwardDiff, P));

Sample

In [6]:
chain, NUTS_tuned = NUTS_init_tune_mcmc(∇P, 1000)

MCMC, adapting ϵ (75 steps)
0.0011 s/step ...done
MCMC, adapting ϵ (25 steps)
0.00067 s/step ...done
MCMC, adapting ϵ (50 steps)
0.0025 s/step ...done
MCMC, adapting ϵ (100 steps)
1.4e-5 s/step ...done
MCMC, adapting ϵ (200 steps)
7.2e-6 s/step ...done
MCMC, adapting ϵ (400 steps)
7.9e-6 s/step ...done
MCMC, adapting ϵ (50 steps)
1.2e-5 s/step ...done
MCMC (1000 steps)
6.7e-6 s/step ...done


(NUTS_Transition{Array{Float64,1},Float64}[NUTS_Transition{Array{Float64,1},Float64}([0.723234], -3.52527, 2, DoubledTurn, 0.990816, 3), NUTS_Transition{Array{Float64,1},Float64}([0.230835], -2.92209, 2, DoubledTurn, 0.990441, 3), NUTS_Transition{Array{Float64,1},Float64}([0.354867], -3.1531, 2, DoubledTurn, 0.965864, 3), NUTS_Transition{Array{Float64,1},Float64}([1.57619], -4.66376, 1, AdjacentTurn, 0.679152, 3), NUTS_Transition{Array{Float64,1},Float64}([1.49542], -4.07015, 1, DoubledTurn, 1.0, 1), NUTS_Transition{Array{Float64,1},Float64}([0.707533], -3.77052, 2, DoubledTurn, 0.996181, 3), NUTS_Transition{Array{Float64,1},Float64}([1.00343], -3.02555, 1, AdjacentTurn, 0.95859, 3), NUTS_Transition{Array{Float64,1},Float64}([0.350429], -3.18228, 1, AdjacentTurn, 0.975989, 3), NUTS_Transition{Array{Float64,1},Float64}([0.332937], -2.8597, 1, DoubledTurn, 0.997217, 1), NUTS_Transition{Array{Float64,1},Float64}([0.339928], -2.86136, 2, DoubledTurn, 1.0, 3)  …  NUTS_Transition{Array{Float

To get the posterior for ``α`` use `get_position` and then transform back.

In [7]:
posterior = TransformVariables.transform.(Ref(problem_transformation(p)), get_position.(chain));

Extract the parameter.

In [8]:
posterior_α = first.(posterior);

check the effective sample size

In [9]:
ess_α = effective_sample_size(posterior_α)

401.79418388192596

NUTS-specific statistics

In [10]:
NUTS_statistics(chain)

Hamiltonian Monte Carlo sample of length 1000
  acceptance rate mean: 0.92, min/25%/median/75%/max: 0.24 0.89 0.97 1.0 1.0
  termination: AdjacentTurn => 36% DoubledTurn => 64%
  depth: 1 => 66% 2 => 34%


check the mean

In [11]:
mean(posterior_α)

0.6359443515023615

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*